In [1]:
import ujson
import pandas as pd

# psycopg2

In [23]:
#import access package to use PostgreSQL directly
import psycopg2
psycopg2.__version__

'2.6.2 (dt dec pq3 ext lo64)'

In [28]:
#Store login credentials in local file to access 
credentials = ujson.load(open("D:\\Python\\posgresql_credentials.json","r"))

In [11]:
#validate user for loaded credentials
credentials['user']

'dev'

In [12]:
#create connection to PostgreSQL Database
conn = psycopg2.connect(host="localhost",
                        database="temp_db_rental", 
                        user=credentials['user'], 
                        password=credentials['password'])

In [13]:
#create cursor object
cur = conn.cursor()

In [14]:
#create SQL statement to execute
cur.execute("SELECT * FROM city LIMIT 10")

In [ ]:
#create iterable that can loop through the selected data
rows = cur.fetchall()

In [15]:
#demo use of iterable
for row in rows:
    print(row)

(1, 'A Corua (La Corua)', 87, datetime.datetime(2006, 2, 15, 9, 45, 25))
(2, 'Abha', 82, datetime.datetime(2006, 2, 15, 9, 45, 25))
(3, 'Abu Dhabi', 101, datetime.datetime(2006, 2, 15, 9, 45, 25))
(4, 'Acua', 60, datetime.datetime(2006, 2, 15, 9, 45, 25))
(5, 'Adana', 97, datetime.datetime(2006, 2, 15, 9, 45, 25))
(6, 'Addis Abeba', 31, datetime.datetime(2006, 2, 15, 9, 45, 25))
(7, 'Aden', 107, datetime.datetime(2006, 2, 15, 9, 45, 25))
(8, 'Adoni', 44, datetime.datetime(2006, 2, 15, 9, 45, 25))
(9, 'Ahmadnagar', 44, datetime.datetime(2006, 2, 15, 9, 45, 25))
(10, 'Akishima', 50, datetime.datetime(2006, 2, 15, 9, 45, 25))


In [16]:
#close the cursor
cur.close()

In [18]:
#close the connection
conn.close()

# SqlAlchemy

In [22]:
import sqlalchemy
sqlalchemy.__version__

'1.1.5'

In [37]:
def pg_connect(user, password, db, host='localhost',port=5432):
    '''Returns a connection and metada SqlAlchemy Object'''
    url = 'postgresql://{}:{}@{}:{}/{}'
    url = url.format(user, password, host, port, db)
    engine = sqlalchemy.create_engine(url, client_encoding = 'utf8')
    meta = sqlalchemy.MetaData()#bind = con, reflect = True)
    
    return (engine, meta)

In [39]:
engine, meta = pg_connect(credentials['user'],
                       credentials['password'],
                       'temp_db_rental')
connection = engine.connect()

In [40]:
print(engine.table_names())

['staff', 'category', 'film_category', 'country', 'actor', 'language', 'inventory', 'payment', 'rental', 'city', 'store', 'film_actor', 'address', 'film', 'customer']


In [41]:
staff_table = sqlalchemy.Table('staff', meta, autoload = True, autoload_with = engine)

In [42]:
print(repr(staff_table))

Table('staff', MetaData(bind=None), Column('staff_id', INTEGER(), table=<staff>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x0000016BEAA2DB70>, for_update=False)), Column('first_name', VARCHAR(length=45), table=<staff>, nullable=False), Column('last_name', VARCHAR(length=45), table=<staff>, nullable=False), Column('address_id', SMALLINT(), ForeignKey('address.address_id'), table=<staff>, nullable=False), Column('email', VARCHAR(length=50), table=<staff>), Column('store_id', SMALLINT(), table=<staff>, nullable=False), Column('active', BOOLEAN(), table=<staff>, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x0000016BEAA470F0>, for_update=False)), Column('username', VARCHAR(length=16), table=<staff>, nullable=False), Column('password', VARCHAR(length=40), table=<staff>), Column('last_update', TIMESTAMP(), table=<staff>, nullable=False, server_default=DefaultClause(<sqlalchemy.sq

In [44]:
repr(meta.tables['staff'])

"Table('staff', MetaData(bind=None), Column('staff_id', INTEGER(), table=<staff>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x0000016BEAA2DB70>, for_update=False)), Column('first_name', VARCHAR(length=45), table=<staff>, nullable=False), Column('last_name', VARCHAR(length=45), table=<staff>, nullable=False), Column('address_id', SMALLINT(), ForeignKey('address.address_id'), table=<staff>, nullable=False), Column('email', VARCHAR(length=50), table=<staff>), Column('store_id', SMALLINT(), table=<staff>, nullable=False), Column('active', BOOLEAN(), table=<staff>, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x0000016BEAA470F0>, for_update=False)), Column('username', VARCHAR(length=16), table=<staff>, nullable=False), Column('password', VARCHAR(length=40), table=<staff>), Column('last_update', TIMESTAMP(), table=<staff>, nullable=False, server_default=DefaultClause(<sqlalchemy.s

In [45]:
stmt = 'SELECT * FROM staff LIMIT 20'

In [46]:
#Result Proxy
res_proxy = connection.execute(stmt)

In [47]:
#Results Set
results = res_proxy.fetchall()

In [48]:
for res in results:
    print(res)

(1, 'Mike', 'Hillyer', 3, 'Mike.Hillyer@sakilastaff.com', 1, True, 'Mike', '8cb2237d0679ca88db6464eac60da96345513964', datetime.datetime(2006, 5, 16, 16, 13, 11, 793280), <memory at 0x0000016BEA9FAB88>)
(2, 'Jon', 'Stephens', 4, 'Jon.Stephens@sakilastaff.com', 2, True, 'Jon', '8cb2237d0679ca88db6464eac60da96345513964', datetime.datetime(2006, 5, 16, 16, 13, 11, 793280), None)


In [50]:
first_row = results[0]
first_row.keys()

['staff_id',
 'first_name',
 'last_name',
 'address_id',
 'email',
 'store_id',
 'active',
 'username',
 'password',
 'last_update',
 'picture']

In [54]:
first_row

(1, 'Mike', 'Hillyer', 3, 'Mike.Hillyer@sakilastaff.com', 1, True, 'Mike', '8cb2237d0679ca88db6464eac60da96345513964', datetime.datetime(2006, 5, 16, 16, 13, 11, 793280), <memory at 0x0000016BEA9FAB88>)